# INIT

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
from utils.logger import get_logger
import config

logger = get_logger(__name__)

# Chain sim setup & start

In [ ]:
from tools.chain_simulator_connector import start_handler
from argparse import Namespace
from time import sleep

docker_path = config.HOME / "Projects/testing/full-stack-docker-compose/chain-simulator"
state_path = config.DEFAULT_WORKSPACE / "states"
args = Namespace(docker_path=str(docker_path), state_path=str(state_path))

chain_sim, found_accounts = start_handler(args)
print(f'Loaded {len(found_accounts)} accounts')
sleep(10)

In [ ]:
from context import Context
from tools.chain_simulator_connector import ChainSimulator

import importlib
import os

os.environ["MX_DEX_ENV"] = "chainsim"
chain_sim = ChainSimulator()
importlib.reload(config)
import config

context = Context()

# Safe price

In [ ]:
from contracts.pair_contract import PairContract
from multiversx_sdk.abi import Abi, AddressValue, U64Value
from multiversx_sdk import SmartContractController, Address

pair_contract: PairContract = context.get_contracts(config.PAIRS_V2)[0]
safe_price_view_contract = context.get_contracts(config.PAIRS_VIEW)[0]

abi = Abi.load(config.HOME / "Projects/dex/mx-exchange-sc/dex/pair/output/safe-price-view.abi.json")
view_controller = SmartContractController(context.network_provider.proxy.get_network_config().chain_id, context.network_provider.proxy, abi)

query = view_controller.create_query(Address.new_from_bech32(safe_price_view_contract.address), "getSafePriceByRoundOffset", 
                                                 [pair_contract.address, 100, [pair_contract.firstToken, 0, 1*10**18]])
response = view_controller.run_query(query)
if response.return_code != "ok":
    raise Exception(f"Error: {response.return_code}")
parsed_response = view_controller.parse_query_response(response)
print(parsed_response)
print(parsed_response[0].amount)

In [ ]:
from multiversx_sdk.abi import BigUIntValue, U64Value, Serializer, StructValue, Field

def get_round_from_observation(raw_entry: bytes):
    first_token_reserve_accumulated = BigUIntValue()
    second_token_reserve_accumulated = BigUIntValue()
    weight_accumulated = U64Value()
    recording_round = U64Value()
    recording_timestamp = U64Value()
    lp_supply_accumulated = BigUIntValue()

    attributes = StructValue([
        Field("first_token_reserve_accumulated", first_token_reserve_accumulated),
        Field("second_token_reserve_accumulated", second_token_reserve_accumulated),
        Field("weight_accumulated", weight_accumulated),
        Field("recording_round", recording_round),
        # Field("recording_timestamp", recording_timestamp),
        # Field("lp_supply_accumulated", lp_supply_accumulated),
    ])
    serializer = Serializer()
    serialized_data = raw_entry.hex()
    serializer.deserialize(serialized_data, [attributes])
    # print(first_token_reserve_accumulated.get_payload())
    # print(second_token_reserve_accumulated.get_payload())
    # print(weight_accumulated.get_payload())
    # print(recording_round.get_payload())
    # print(recording_timestamp.get_payload())
    # print(lp_supply_accumulated.get_payload())

    return recording_round.get_payload()

## monitor state observations

In [ ]:
from multiversx_sdk import ProxyNetworkProvider

def get_pair_state(pair_contract: PairContract):
    if config.CURRENT_ENV.value == "mainnet":
        proxy = ProxyNetworkProvider(config.HISTORY_PROXY)
    else:
        proxy = context.network_provider.proxy
    state = proxy.get_account_storage(Address.new_from_bech32(pair_contract.address))
    return state

def get_last_observation_round(pair_contract: PairContract):
    latest_round = 0
    state = get_pair_state(pair_contract)
    for entry in state.entries:
        if "price_observations.item" in entry.key:
            round = get_round_from_observation(entry.value)
            if round > latest_round:
                latest_round = round

    return latest_round

# Upgrade

Safeprice view

In [ ]:
pair_view_contract: PairContract = context.get_contracts(config.PAIRS_VIEW)[0]
pair_view_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.HOME / "Projects/dex/mx-exchange-sc/dex/pair/output/safe-price-view.wasm", [], True)
chain_sim.advance_blocks(1)

Template

In [ ]:
from contracts.router_contract import RouterContract
from contracts.pair_contract import PairContract

bytecode_path = config.HOME / "Projects/dex/mx-exchange-sc/dex/pair/output/pair.wasm"
router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]
template_pair_address = router_contract.get_pair_template_address(context.network_provider.proxy)
print(template_pair_address)
template_pair = PairContract.load_contract_by_address(template_pair_address)

# upgrade template pair contract
template_pair.contract_upgrade(context.deployer_account, context.network_provider.proxy, bytecode_path, [], no_init=True)
chain_sim.advance_blocks(1)

Pair

In [ ]:
from contracts.router_contract import RouterContract
router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]
router_contract.pair_contract_upgrade(context.deployer_account, context.network_provider.proxy, [pair_contract.firstToken, pair_contract.secondToken])
chain_sim.advance_blocks(1)

Router

In [ ]:
from contracts.router_contract import RouterContract
router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]
router_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.HOME / "Projects/dex/mx-exchange-sc/dex/router/output/router.wasm")
chain_sim.advance_blocks(1)
router_contract.resume(context.deployer_account, context.network_provider.proxy)
chain_sim.advance_blocks(1)

New observation interval setup

In [ ]:
from contracts.router_contract import RouterContract
router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]
router_contract.set_safe_price_round_save_interval(context.deployer_account, context.network_provider.proxy, [6, [pair_contract.address]])
chain_sim.advance_blocks(1)

# SWAPS

In [ ]:
print(f"firstToken: {pair_contract.firstToken}")
print(f"secondToken: {pair_contract.secondToken}")

In [ ]:
from contracts.pair_contract import SwapFixedInputEvent

event = SwapFixedInputEvent(pair_contract.firstToken, 1000,
                            pair_contract.secondToken, 1)
pair_contract.swap_fixed_input(context.network_provider, context.deployer_account, event)
chain_sim.advance_blocks(1)

snipe the observation round

In [ ]:
from time import sleep
from utils.contract_retrievers import PairContractDataFetcher
from contracts.pair_contract import SwapFixedInputEvent
from multiversx_sdk.abi import TokenIdentifierValue, BigUIntValue

def snipe_observation_round():
    amount_to_swap = 1000 * 10**18
    while True:
        last_observation_round = get_last_observation_round(pair_contract)
        print(f"current round: {context.network_provider.get_round()}")
        print(f"latest observation round: {last_observation_round}")

        observation_interval = pair_contract.get_safe_price_round_save_interval(context.network_provider.proxy)
        current_round = context.network_provider.get_round()
        next_observation_round = current_round + observation_interval - (current_round - last_observation_round) % observation_interval

        # wait until next observation round
        print(f"targetting observation round {next_observation_round}")
        while context.network_provider.get_round() < next_observation_round - 2:
            sleep(0.2)
            # chain_sim.advance_blocks(1)

        pair_data_fetcher = PairContractDataFetcher(Address.new_from_bech32(pair_contract.address), context.network_provider.proxy.url)
        amount_out = pair_data_fetcher.get_data("getAmountOut",[TokenIdentifierValue(pair_contract.firstToken), BigUIntValue(amount_to_swap)])

        event_forwards = SwapFixedInputEvent(pair_contract.firstToken, amount_to_swap,
                                    pair_contract.secondToken, 1)
        pair_contract.swap_fixed_input(context.network_provider, context.deployer_account, event_forwards)
        
        print(f"swaps sent in round: {context.network_provider.get_round()}")
        # TODO: to remove if using chain sim manual blocks advance
        sleep(1)

        event_consolidation = SwapFixedInputEvent(pair_contract.firstToken, 1001,
                                    pair_contract.secondToken, 1)
        pair_contract.swap_fixed_input(context.network_provider, context.deployer_account, event_consolidation)

        sleep(1)

        event_backwards = SwapFixedInputEvent(pair_contract.secondToken, amount_out,
                                    pair_contract.firstToken, 1)
        pair_contract.swap_fixed_input(context.network_provider, context.deployer_account, event_backwards)
        
        # chain_sim.advance_blocks(1)

In [ ]:
snipe_observation_round()

Random swaps

In [ ]:
from utils.utils_chain import get_all_token_nonces_details_for_account
from utils.contract_retrievers import PairContractDataFetcher
from contracts.pair_contract import SwapFixedInputEvent
from time import sleep
def random_swaps():
    import random
    spend_range_percent = [10, 30]
    swaps_delay_ms = [200, 40000]
    while True:
        # select a random token to spend
        spend_token = random.choice([pair_contract.firstToken, pair_contract.secondToken])
        other_token = pair_contract.firstToken if spend_token == pair_contract.secondToken else pair_contract.secondToken
        
        # get the balance of the spent token
        response = get_all_token_nonces_details_for_account(spend_token, context.deployer_account.address.bech32(), context.network_provider.proxy)[0]
        balance = int(response.get("balance", 0))
        if balance == 0:
            logger.warning(f"No balance for {spend_token}")
        spend_amount = random.randint(spend_range_percent[0], spend_range_percent[1]) * balance // 100

        # determine delay between swaps
        delay = random.randint(swaps_delay_ms[0], swaps_delay_ms[1])
        logger.debug(f"Swapping {spend_amount} {spend_token} for {other_token}. Delay: {delay}ms")
        
        sleep(delay / 1000)

        # prepare swap
        event = SwapFixedInputEvent(spend_token, spend_amount, other_token, 1)
        pair_contract.swap_fixed_input(context.network_provider, context.deployer_account, event)
        

In [ ]:
random_swaps()

Manual checks

In [ ]:
state = get_pair_state(pair_contract)
for entry in state.entries:
    if "price" in entry.key:
        print(f"{entry.key}: {entry.raw}")

In [ ]:
from enum import Enum
from typing import Any
class Timebase(Enum):
    ROUND = "Round"
    TIMESTAMP = "Timestamp"

def get_lp_safe_price_by_offset(timebase: Timebase, context: Context, abi: Abi, pair_contract: PairContract, 
                                offset: int, reference_amount: int) -> list[Any]:
    """ Returns a list of namespaces containing the two tokens underlying the given lp reference amount.
        Each namespace contains:
        - token_identifier
        - nonce
        - amount"""
    safe_price_view_contract = context.get_contracts(config.PAIRS_VIEW)[0]
    view_controller = SmartContractController(context.network_provider.proxy.get_network_config().chain_id, context.network_provider.proxy, abi)
    
    endpoint = f"getLpTokensSafePriceBy{timebase.value}Offset"
    query = view_controller.create_query(Address.new_from_bech32(safe_price_view_contract.address), endpoint, 
                                                 [pair_contract.address, offset, reference_amount])
    response = view_controller.run_query(query)
    if response:
        return view_controller.parse_query_response(response)
    return []

tokens = get_lp_safe_price_by_offset(Timebase.ROUND, context, abi, pair_contract, 100, 1 * 10**18)
print([(t.token_identifier, t.amount) for t in tokens])

# SET STATE for user

In [ ]:
user = "erd1pye4dsy3fs956skp8wgf5pjtfynvw5y52gwdmsrrum9jcv3y2vusk7we6t"
chain_sim.apply_states([[{
        "address": user,
        "nonce": 0,
        "balance": "100000000000000000000",
        "username": "",
        "code": "",
        "developerReward": "0",
        "ownerAddress": "",
        "pairs": {}
    }]])

In [ ]:
user = "erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97"
esdt = "WEGLD-bd4d79"
amount = 10000 * 10**18

from utils.utils_chain import dec_to_padded_hex
current_entry = context.network_provider.proxy.get_account_storage_entry(Address.new_from_bech32(user), f"ELRONDesdt{esdt}")
if not current_entry:
    raise Exception("No entry found")

print(current_entry.value.hex())
print(dec_to_padded_hex(amount))
header = current_entry.value.hex()[:2]
new_entry = f"{header}{dec_to_padded_hex(len(dec_to_padded_hex(amount)) // 2 + 1)}{'00'}{dec_to_padded_hex(amount)}"
print(new_entry)

chain_sim.apply_states([[{
        "address": user,
        "pairs": {
            current_entry.key.encode().hex(): new_entry
        }
    }]])

# 120600174876e800
# 12070038f9f643173a
# 1208003877543179f11e
# 120900d02ab486cedc0000
# 120900059309ed4f0a1e0d


# FEES setup for pairs

In [ ]:
from contracts.pair_contract import PairContract
from utils.contract_retrievers import retrieve_pair_by_address

pair_contract: PairContract
pair_contract = retrieve_pair_by_address("erd1qqqqqqqqqqqqqpgqph6g8569lnvpgd3x569hd6n6qse2aw0w0n4sms6nzv")    # operating pair
mex_contract = retrieve_pair_by_address("erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2") # egldmex contract

In [ ]:
from contracts.pair_contract import PairContract
from utils.contract_retrievers import retrieve_pair_by_address

pair_contract: PairContract
pair_contract = retrieve_pair_by_address("erd1qqqqqqqqqqqqqpgq0e9pmlzr0nk5nkulzcmessttsjkzr4xf0n4sue4r8e")    # operating pair

In [ ]:
# whitelist in egldmex pair for swap no fees
mex_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, pair_contract.address)

In [ ]:
# set where to swap and what to do with the fees
pair_contract.add_trusted_swap_pair(context.deployer_account, context.network_provider.proxy,
                                    [
                                        mex_contract.address,
                                        mex_contract.firstToken,
                                        mex_contract.secondToken
                                    ])

In [ ]:
pair_contract.set_fees_percents(context.deployer_account, context.network_provider.proxy,
                                [300, 100])

In [ ]:
from contracts.router_contract import RouterContract
router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]

pair_contract.set_fee_on_via_router(context.deployer_account, context.network_provider.proxy, router_contract, 
                                [
                                    config.ZERO_CONTRACT_ADDRESS,
                                    mex_contract.secondToken
                                ])

In [ ]:
from contracts.pair_contract import AddLiquidityEvent

event = AddLiquidityEvent(pair_contract.firstToken, 127791780000000000000, 1, pair_contract.secondToken, 5000000000000000000, 1)
pair_contract.add_liquidity(context.network_provider, context.deployer_account, event)

In [ ]:
pair_contract.resume(context.deployer_account, context.network_provider.proxy)